In [13]:
!pip install -q -U google-generativeai python-dotenv

I0000 00:00:1722406206.810522 6006664 work_stealing_thread_pool.cc:320] WorkStealingThreadPoolImpl::PrepareFork


In [15]:
import google.generativeai as genai
from IPython.display import Markdown, Image

In [24]:
import os, json
from dotenv import load_dotenv

load_dotenv()
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [69]:
actual_data = {
    "images/1.png": {
        "calories": 357.81,
        "protein": 20.00,
        "carbs": 20.96,
        "fat": 21.69
    },
    "images/2.png": {
        "calories": 655.99,
        "protein": 93.24,
        "carbs": 23.34,
        "fat": 20.22
    },
    "images/3.png": {
        "calories": 351.89,
        "protein": 12.10,
        "carbs": 57.50,
        "fat": 9.77
    },
    "images/4.png": {
        "calories": 184.99,
        "protein": 24.54,
        "carbs": 8.90,
        "fat": 6.14
    }
}

In [82]:
def get_nutrition_info(image_path):
    # Upload the file
    sample_file = genai.upload_file(path=image_path)

    # Run Gemini model
    model = genai.GenerativeModel(model_name="gemini-1.5-pro")
    prompt = """
    You are an expert nutritionist with extensive experience in visual food analysis and portion estimation. Your task is to provide the most accurate nutrition information possible for the given food image.

    Instructions:
    1. Carefully analyze the image, considering that it was taken 50 cm above the food item.
    2. Identify all visible food items and ingredients.
    3. Estimate the portion size of each item, taking into account the known distance from the camera.
    4. Consider the preparation method (e.g., fried, grilled, raw) as it affects nutritional content.
    5. Factor in any visible condiments, sauces, or dressings.
    6. Use your expertise to estimate the most likely recipe or preparation method if not immediately apparent.
    7. Calculate the total nutritional content based on your analysis, considering:
       - Calories (kcal)
       - Protein (g)
       - Carbohydrates (g)
       - Fat (g)
    8. Round your estimates to the nearest whole number for simplicity.
    9. If you're uncertain about any aspect, provide your best estimate and indicate the level of confidence.

    Output your nutritional analysis in the following JSON format, ensuring all values are integers:
    {
        "calories": <calories>,
        "protein": <protein>,
        "carbs": <carbs>,
        "fat": <fat>
    }

    Remember, accuracy is crucial. Use all available visual cues and your nutritional expertise to provide the most precise estimates possible.
    """
    response = model.generate_content([sample_file, prompt])

    return response.text

In [83]:
def get_percent_error(image_path):    
    actual_values = actual_data[image_path]
    predicted_values = json.loads(get_nutrition_info(image_path))
    
    output = {}
    for key in actual_values:
        actual = actual_values[key]
        predicted = predicted_values[key]
        percent_error = abs((predicted - actual) / actual) * 100
        output[key] = {
            "actual": actual,
            "predicted": predicted,
            "percent_error": round(percent_error, 2)
        }
    
    return output

In [84]:
percent_error_1 = get_percent_error("images/1.png")
percent_error_1

{'calories': {'actual': 357.81, 'predicted': 450, 'percent_error': 25.77},
 'protein': {'actual': 20.0, 'predicted': 20, 'percent_error': 0.0},
 'carbs': {'actual': 20.96, 'predicted': 40, 'percent_error': 90.84},
 'fat': {'actual': 21.69, 'predicted': 25, 'percent_error': 15.26}}

In [85]:
percent_error_2 = get_percent_error("images/2.png")
percent_error_2

{'calories': {'actual': 655.99, 'predicted': 770, 'percent_error': 17.38},
 'protein': {'actual': 93.24, 'predicted': 75, 'percent_error': 19.56},
 'carbs': {'actual': 23.34, 'predicted': 40, 'percent_error': 71.38},
 'fat': {'actual': 20.22, 'predicted': 45, 'percent_error': 122.55}}

In [86]:
percent_error_3 = get_percent_error("images/3.png")
percent_error_3

{'calories': {'actual': 351.89, 'predicted': 370, 'percent_error': 5.15},
 'protein': {'actual': 12.1, 'predicted': 10, 'percent_error': 17.36},
 'carbs': {'actual': 57.5, 'predicted': 60, 'percent_error': 4.35},
 'fat': {'actual': 9.77, 'predicted': 12, 'percent_error': 22.82}}

In [89]:
percent_error_4 = get_percent_error("images/4.png")
percent_error_4

{'calories': {'actual': 184.99, 'predicted': 400, 'percent_error': 116.23},
 'protein': {'actual': 24.54, 'predicted': 35, 'percent_error': 42.62},
 'carbs': {'actual': 8.9, 'predicted': 30, 'percent_error': 237.08},
 'fat': {'actual': 6.14, 'predicted': 18, 'percent_error': 193.16}}

### Really bad for mixed food items ^^^

In [64]:
print(get_nutrition_info("images/5.jpeg"))

{
        "calories": 700,
        "protein": 40,
        "carbs": 90,
        "fat": 25
    }



In [67]:
print(get_nutrition_info("images/6.jpeg"))

{
    "calories": 700,
    "protein": 45,
    "carbs": 80,
    "fat": 25
}



In [68]:
print(get_nutrition_info("images/7.jpeg"))

{
    "calories": 60,
    "protein": 0,
    "carbs": 15,
    "fat": 0
}

